In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

sub_ids = test['portfolio_id']
test = test.drop('portfolio_id', axis=1)
y = train['return']
train = train.drop(['portfolio_id', 'return'], axis=1)

# remove some  fields
train = train.drop(['start_date', 'creation_date', 'sell_date', 'indicator_code', 'status', 'desk_id'], axis=1)
test = test.drop(['start_date', 'creation_date', 'sell_date', 'indicator_code', 'status', 'desk_id'], axis=1)

# handle missing values 
train['hedge_value'].fillna(False, inplace=True)
test['hedge_value'].fillna(False, inplace=True)
# missing values for numeric fields
train['sold'].fillna(train['sold'].median(), inplace=True)
train['bought'].fillna(train['bought'].median(), inplace=True)
train['libor_rate'].fillna(train['libor_rate'].median(), inplace=True)
test['libor_rate'].fillna(train['libor_rate'].median(), inplace=True)

# encode categorical fields
obj_cols = [x for x in train.columns if train[x].dtype == 'object']
encoder = LabelEncoder()
for x in obj_cols:
    encoder.fit(train[x])
    train[x] = encoder.transform(train[x])
    test[x] = encoder.transform(test[x])

In [34]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=7)
scores = cross_val_score(forest_reg, train, y, scoring='r2', cv=5)
print(scores)
print('mean r2:',np.mean(scores))

[-0.99699912  0.97393782  0.89965033  0.38801503  0.81863494]
mean r2: 0.416647798489


In [35]:
np.mean(scores)

0.41664779848853933

In [36]:
from IPython.display import FileLink

forest_reg = RandomForestRegressor(random_state=7)
forest_reg.fit(train, y)
preds = forest_reg.predict(test)
sub = pd.DataFrame({'portfolio_id': sub_ids, 'return': preds})
filename = 'sub_returns.csv'
sub.to_csv(filename, index=False)
FileLink(filename)  # lb 0.94277

/home/shinto/hackerearth/socgen/annual_returns/source/sub_returns.csv